### Векторные представления слов (word embeddings)

In [5]:
data = "/content/kinopoisk-top250.csv"

In [6]:
import pandas as pd

In [7]:
df = pd.read_csv(data)
df.head()

,rating,movie,year,country,rating_ball,overview,director,screenwriter,actors,url_logo
0,0,Побег из Шоушенка,1994,США,9.111,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Тим Роббинс; Морган Фриман; Боб Гантон; Уильям...,'https://st.kp.yandex.net/images/film_iphone/i...
1,1,Зеленая миля,1999,США,9.062,Пол Эджкомб — начальник блока смертников в тюр...,Фрэнк Дарабонт,Фрэнк Дарабонт; Стивен Кинг,Том Хэнкс; Дэвид Морс; Бонни Хант; Майкл Кларк...,'https://st.kp.yandex.net/images/film_iphone/i...
2,2,Форрест Гамп,1994,США,8.913,От лица главного героя Форреста Гампа; слабоум...,Роберт Земекис,Эрик Рот; Уинстон Грум,Том Хэнкс; Робин Райт; Салли Филд; Гэри Синиз;...,'https://st.kp.yandex.net/images/film_iphone/i...
3,3,Список Шиндлера,1993,США,8.817,Фильм рассказывает реальную историю загадочног...,Стивен Спилберг,Стивен Зеллиан; Томас Кенилли,Лиам Нисон; Бен Кингсли; Рэйф Файнс; Кэролайн ...,'https://st.kp.yandex.net/images/film_iphone/i...
4,4,1+1,2011,Франция,8.807,Пострадав в результате несчастного случая; бог...,Оливье Накаш; Эрик Толедано,Оливье Накаш; Филипп Поццо ди Борго; Эрик Т...,Франсуа Клюзе; Омар Си; Анн Ле Ни; Одри Флеро;...,'https://st.kp.yandex.net/images/film_iphone/i...


In [8]:
!pip install pymorphy2
!pip install stop_words
!pip install annoy

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 10.4 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=3383883b9f4601f49a96b2b763c474b8c87c6f7e1473f42ea1f9f2442060a631
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 646 kB 7.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391672 sha256=f5a38a5bdff4ecc55f6cb34a67f081d5628c6a8a5b90ca0a0ed0f973dcead6fc
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [9]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import numpy as np

In [10]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [11]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

In [12]:
for line in df['overview']:
    spls = preprocess_txt(line)
    sentences.append(spls)
    c += 1
    if c > 100000:
       break

In [13]:
sentences = [i for i in sentences if len(i) > 2]

In [14]:
sentences[0]

['бухгалтер',
 'энди',
 'дюфрейн',
 'обвинить',
 'убийство',
 'собственный',
 'жена',
 'любовник',
 'оказаться',
 'тюрьма',
 'название',
 'шоушенк',
 'сталкиваться',
 'жестокость',
 'беззаконие',
 'царить',
 'сторона',
 'решётка',
 'попадать',
 'стена',
 'становиться',
 'раб',
 'конец',
 'энди',
 'обладать',
 'живой',
 'ум',
 'добрый',
 'душа',
 'находить',
 'подход',
 'заключить',
 'охранник',
 'добиваться',
 'особый',
 'расположение']

In [15]:
modelW2V = Word2Vec(sentences=sentences, size=50, window=5, min_count=1)

In [16]:
modelFT = FastText(sentences=sentences, size=50, min_count=1, window=5)

In [21]:
df['vector_w2v'] = 0
df['vector_ft'] = 0

In [104]:
df.tail()

,rating,movie,year,country,rating_ball,overview,director,screenwriter,actors,url_logo,vector_w2v,vector_ft
245,245,Крамер против Крамера,1979,США,7.984,Тед Крамер; целиком посвятив себя карьере; не ...,Роберт Бентон,Роберт Бентон; Эйвери Кормэн,Дастин Хоффман; Мэрил Стрип; Джейн Александр; ...,'https://st.kp.yandex.net/images/film_iphone/i...,0,0
246,246,Пираты Карибского моря: На краю Света,2007,США,8.007,Новые приключения Джека Воробья и его друзей У...,Гор Вербински,Тед Эллиот; Терри Россио; Стюарт Битти; ...,Джонни Депп; Джеффри Раш; Орландо Блум; Кира Н...,'https://st.kp.yandex.net/images/film_iphone/i...,0,0
247,247,Иди и смотри,1985,СССР,8.077,Флера — шестнадцатилетний мальчишка; откопавши...,Элем Климов,Алесь Адамович; Элем Климов,Алексей Кравченко; Ольга Миронова; Любомирас Л...,'https://st.kp.yandex.net/images/film_iphone/i...,0,0
248,248,Профессионал,1981,Франция,8.112,Убить президента африканской республики. Такое...,Жорж Лотнер,Мишель Одиар; Патрик Александер; Жорж Лотне...,Жан-Поль Бельмондо; Жан Дезайи; Сирил Клер; Ма...,'https://st.kp.yandex.net/images/film_iphone/i...,0,0
249,249,Холодное лето пятьдесят третьего...,1987,СССР,8.050,Двое бывших политзаключённых; недавно вышедших...,Александр Прошкин,Эдгар Дубровский,Валерий Приёмыхов; Анатолий Папанов; Виктор Ст...,'https://st.kp.yandex.net/images/film_iphone/i...,0,0


In [107]:
w2v_index = annoy.AnnoyIndex(50 ,'angular')
ft_index = annoy.AnnoyIndex(50 ,'angular')
counter = 0
for line in df['overview']:
  n_w2v = 0
  n_ft = 0
  vector_w2v = np.zeros(50)
  vector_ft = np.zeros(50)
  for ind in range (len(df)):
    for el in df['overview'][ind]:
      if el in modelW2V:
        vector_w2v += modelW2V[el]
        n_w2v += 1
      if el in modelFT:
        vector_ft += modelFT[el]
        n_ft += 1
    if n_w2v > 0:
      vector_w2v = vector_w2v / n_w2v
      #df['vector_w2v'][ind] = vector_w2v
    if n_ft > 0:
      vector_ft = vector_ft / n_ft
      #df['vector_ft'][ind] = vector_ft
  w2v_index.add_item(counter, vector_w2v)
  ft_index.add_item(counter, vector_ft)
  counter +=1

w2v_index.build(10)
ft_index.build(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


True

In [27]:
import re

In [113]:
d = {}
for ind in range (len(df)):
  d[df['movie'][ind]] = df['rating'][ind]

In [114]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [115]:
def find_similar(index, film, n=3):
  if film in d.keys():
    ind = d[film]
    result = index.get_nns_by_vector(index.get_item_vector(ind), n=3)
    list_of_films = []
    for el in result:
      k = get_key(d, el)
      list_of_films.append(k)
    return list_of_films 
  else:
    print("No such a film")
  

In [117]:
find_similar(w2v_index, '1+1 ')

['Форрест Гамп ', 'Бриллиантовая рука ', 'Хатико: Самый верный друг ']